In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo

In [2]:
flight_nr = "11"

In [3]:
# Path templates for GPS and VO data
vo_data_template = "/mnt/hdd02/ba_copter_videos/Flight_{}/rotated_vo_{}.traj"
gps_data_path_template = "/mnt/hdd02/ba_copter_videos/Flight_{}/gps_{}_interpolated.traj"

# Set the file paths for VO and GPS data based on the flight number
vo_data_path = vo_data_template.format(flight_nr, flight_nr)
gps_data_path = gps_data_path_template.format(flight_nr, flight_nr)

# Load the VO and GPS data from CSV files
vo_data = pd.read_csv(vo_data_path, header=None)
gps_data = pd.read_csv(gps_data_path, header=None)

In [4]:
# Geschwindigkeiten berechnen
def calculate_velocity(df):
    velocities = []
    for i in range(1, len(df)):
        t1, x1, y1, z1, q1, q2, q3, q4 = df.iloc[i-1]
        t2, x2, y2, z2, q5, q6, q7, q8 = df.iloc[i]
        dt = t2 - t1
        if dt == 0:
            continue
        vx = (x2 - x1) / dt
        vy = (y2 - y1) / dt
        vz = (z2 - z1) / dt
        velocities.append((t2, x1, y1, z1, vx, vy, vz))
    return velocities

In [5]:
velocities = calculate_velocity(vo_data)

# 3D-Plot erstellen
fig = go.Figure()

for i, (t, x, y, z, vx, vy, vz) in enumerate(velocities):
    if i % 25 != 0:  
        continue
    fig.add_trace(go.Cone(
        x=[x], y=[y], z=[z],
        u=[vx], v=[vy], w=[vz],
        sizemode="absolute",  # Set size mode to absolute
        sizeref=0.8,  # Adjust this value to make the cones narrower
        anchor="tail",
        colorscale='Blues'
    ))

fig.update_layout(
    title=f'3D Velocity Vectors',
    showlegend=False
)

# Adjust layout
fig.update_scenes(xaxis=dict(title='Latitude'),
                  yaxis=dict(title='Longitude', autorange='reversed'),
                  zaxis=dict(title='Altitude [m]'))

# Interaktiven Plot anzeigen
pyo.plot(fig)

'temp-plot.html'